In [1]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from numpy import pi

In [51]:
def load(qc, index, indexAux,  f, d, fTotal, dTotal, n, iLength):
    # ccx for every bit that requires a 1
    qc.cx(index, indexAux)
    qc.x(indexAux)
    
    for i in range(0, 2**iLength):
        gates = []
        copy = i
        for j in range(iLength-1, -1, -1):
            if (copy - 2**j >= 0):
                copy -= 2**j
                gates.append(index[j])
            else:
                gates.append(indexAux[j])
        for l in range(0, n):
            gates.append(fTotal[i][l])
            qc.mct(gates, f[l])
            gates.pop()
            gates.append(dTotal[i][l])
            qc.mct(gates, d[l])
            gates.pop()
        qc.barrier()
    
    qc.x(indexAux)           
    qc.cx(index, indexAux)
        
def initilizeQubits(qc, a, b, regA, regB, n):
    for i in range(n):
        if (a[i] == "1"):
            qc.x(regA[n-(i+1)])
        if (b[i] == "1"):
            qc.x(regB[n-(i+1)])

In [59]:
f0 = "11"
f1 = "00"
# f2 = "10"
# f3 = "01"

d0 = "11"
d1 = "00"
# d2 = "10"
# d3 = "01"

iLength = 1
n = 2
index = QuantumRegister(iLength, name="i")
indexAux = QuantumRegister(iLength, name="iAux")
f = QuantumRegister(n, name="f")
d = QuantumRegister(n, name="d")

fTotal = []
dTotal = []

fTotal.append(QuantumRegister(n, name="f0"))
fTotal.append(QuantumRegister(n, name="f1"))
# fTotal.append(QuantumRegister(n, name="f2"))
# fTotal.append(QuantumRegister(n, name="f3"))

dTotal.append(QuantumRegister(n, name="d0"))
dTotal.append(QuantumRegister(n, name="d1"))
# dTotal.append(QuantumRegister(n, name="d2"))
# dTotal.append(QuantumRegister(n, name="d3"))

output = ClassicalRegister(n*2+iLength)

# qc = QuantumCircuit(index, indexAux,  f, d, fTotal[0], fTotal[1], fTotal[2], fTotal[3],
#                     dTotal[0], dTotal[1], dTotal[2], dTotal[3], output)

qc = QuantumCircuit(index, indexAux, f, d, fTotal[0], fTotal[1], 
                    dTotal[0], dTotal[1], output)
qc.h(index)
initilizeQubits(qc, f0, f1, fTotal[0], fTotal[1], n)
initilizeQubits(qc, d0, d1, dTotal[0], dTotal[1], n)
# initilizeQubits(qc, f2, f3, fTotal[2], fTotal[3], n)
# initilizeQubits(qc, d2, d3, dTotal[2], dTotal[3], n)

load(qc, index, indexAux, f, d, fTotal, dTotal, n, iLength)


qc.measure(index, output[0:iLength])
qc.measure(f, output[iLength:iLength+n])
qc.measure(d, output[iLength+n:iLength+2*n])
qc.draw()


┌───┐                               ░                      ░           »
    i: ┤ H ├──■────────────────────────────░───■────■────■────■───░────────■──»
       └───┘┌─┴─┐┌───┐                     ░   │    │    │    │   ░ ┌───┐┌─┴─┐»
 iAux: ─────┤ X ├┤ X ├──■────■────■────■───░───┼────┼────┼────┼───░─┤ X ├┤ X ├»
            └───┘└───┘┌─┴─┐  │    │    │   ░ ┌─┴─┐  │    │    │   ░ └┬─┬┘└───┘»
  f_0: ───────────────┤ X ├──┼────┼────┼───░─┤ X ├──┼────┼────┼───░──┤M├──────»
                      └─┬─┘  │  ┌─┴─┐  │   ░ └─┬─┘  │  ┌─┴─┐  │   ░  └╥┘  ┌─┐ »
  f_1: ─────────────────┼────┼──┤ X ├──┼───░───┼────┼──┤ X ├──┼───░───╫───┤M├─»
                        │  ┌─┴─┐└─┬─┘  │   ░   │  ┌─┴─┐└─┬─┘  │   ░   ║   └╥┘ »
  d_0: ─────────────────┼──┤ X ├──┼────┼───░───┼──┤ X ├──┼────┼───░───╫────╫──»
                        │  └─┬─┘  │  ┌─┴─┐ ░   │  └─┬─┘  │  ┌─┴─┐ ░   ║    ║  »
  d_1: ─────────────────┼────┼────┼──┤ X ├─░───┼────┼────┼──┤ X ├─░───╫────╫──»
       ┌───┐            │    │    │  └─┬─┘ ░   │    │    │  └─┬─┘ ░   ║    ║  »
 f0_0: ┤ X ├────────────■────┼────┼────┼───░───┼────┼────┼────┼───░───╫────╫──»
       ├───┤                 │    │    │   ░   │    │    │    │   ░   ║    ║  »
 f0_1: ┤ X ├─────────────────┼────■────┼───░───┼────┼────┼────┼───░───╫────╫──»
       └───┘                 │         │   ░   │    │    │    │   ░   ║    ║  »
 f1_0: ──────────────────────┼─────────┼───░───■────┼────┼────┼───░───╫────╫──»
                             │         │   ░        │    │    │   ░   ║    ║  »
 f1_1: ──────────────────────┼─────────┼───░────────┼────■────┼───░───╫────╫──»
       ┌───┐                 │         │   ░        │         │   ░   ║    ║  »
 d0_0: ┤ X ├─────────────────■─────────┼───░────────┼─────────┼───░───╫────╫──»
       ├───┤                           │   ░        │         │   ░   ║    ║  »
 d0_1: ┤ X ├───────────────────────────■───░────────┼─────────┼───░───╫────╫──»
       └───┘                               ░        │         │   ░   ║    ║  »
 d1_0: ────────────────────────────────────░────────■─────────┼───░───╫────╫──»
                                           ░                  │   ░   ║    ║  »
 d1_1: ────────────────────────────────────░──────────────────■───░───╫────╫──»
                                           ░                      ░   ║    ║  »
c26: 5/═══════════════════════════════════════════════════════════════╩════╩══»
                                                                      1    2  »
«             ┌─┐
«    i: ──────┤M├
«             └╥┘
« iAux: ───────╫─
«              ║ 
«  f_0: ───────╫─
«              ║ 
«  f_1: ───────╫─
«       ┌─┐    ║ 
«  d_0: ┤M├────╫─
«       └╥┘┌─┐ ║ 
«  d_1: ─╫─┤M├─╫─
«        ║ └╥┘ ║ 
« f0_0: ─╫──╫──╫─
«        ║  ║  ║ 
« f0_1: ─╫──╫──╫─
«        ║  ║  ║ 
« f1_0: ─╫──╫──╫─
«        ║  ║  ║ 
« f1_1: ─╫──╫──╫─
«        ║  ║  ║ 
« d0_0: ─╫──╫──╫─
«        ║  ║  ║ 
« d0_1: ─╫──╫──╫─
«        ║  ║  ║ 
« d1_0: ─╫──╫──╫─
«        ║  ║  ║ 
« d1_1: ─╫──╫──╫─
«        ║  ║  ║ 
«c26: 5/═╩══╩══╩═
«        3  4  0

In [68]:
# Measure
aer_sim = Aer.get_backend('aer_simulator')
transpiled_grover_circuit = transpile(qc, aer_sim)
qobj = assemble(transpiled_grover_circuit)
results = aer_sim.run(qobj).result()
counts = results.get_counts()
print(counts)

{'000001': 531, '010010': 493}


In [63]:
def adder(qc, regA, regB, regC, outputReg, n):
    qc.barrier()
    for i in range(n):
        
        if (i == 0):
            addBits(qc, regA, regB, outputReg, i)
        else:
            addBits(qc, regA, regB, outputReg, i, regC, i-1)
        getCarryBits(qc, regA, regB, regC, i)
    addOnCarry(qc, regC, outputReg, n-1, n)
    
    for i in range(n):
        getCarryBits(qc, regA, regB, regC, i)
    qc.barrier()
    
def addBits(qc, regA, regB, outputReg, i, regC=None, j=None):
    if (regC == None):
        qc.cx(regA[i], outputReg[i])
        qc.cx(regB[i], outputReg[i])
    else:
        qc.cx(regA[i], outputReg[i])
        qc.cx(regB[i], outputReg[i])
        qc.cx(regC[j], outputReg[i])
            
        
def getCarryBits(qc, regA, regB, regC, i):
    qc.ccx(regA[i], regB[i], regC[i])
    qc.ccx(regA[i], regC[i-1], regC[i])
    qc.ccx(regB[i], regC[i-1], regC[i])
    
def addOnCarry(qc, regC, outputReg, cIndex, oIndex):
    qc.cx(regC[cIndex], outputReg[oIndex])
    
def getCarryBitsCumulative(qc, regA, regB, regC, n):
    for i in range(n-1):
        qc.ccx(regA[i], regB[i], regC[i+1])
        qc.cx(regA[i], regB[i])
        qc.ccx(regC[i], regB[i], regC[i+1])
        
    qc.ccx(regA[n-1], regB[n-1], regB[n])
    qc.cx(regA[n-1], regB[n-1])
    qc.ccx(regC[n-1], regB[n-1], regB[n])
    
def reverseCarryBitsCummulative(qc, regA, regB, regC, n):
    #Reversing the gate operation performed on b[n-1]
    qc.cx(regC[n-1], regB[n-1])
    #Reversing the gate operations performed during the carry gate implementations
    #This is done to ensure the sum gates are fed with the correct input bit states
    for i in range(n-1):
        qc.ccx(regC[(n-2)-i], regB[(n-2)-i], regC[(n-1)-i])
        qc.cx(regA[(n-2)-i], regB[(n-2)-i])
        qc.ccx(regA[(n-2)-i], regB[(n-2)-i], regC[(n-1)-i])
        #These two operations act as a sum gate; if a control bit is at                
        #the 1> state then the target bit b[(n-2)-i] is flipped
        qc.cx(regC[(n-2)-i], regB[(n-2)-i])
        qc.cx(regA[(n-2)-i], regB[(n-2)-i])
def cummulativeAdder(qc, regA, regC, output, n):
    qc.barrier()
    
    print(n, )
    getCarryBitsCumulative(qc, regA, output, regC, n)
    reverseCarryBitsCummulative(qc, regA, output, regC, n)
    qc.barrier()
    
def initilizeQubits(qc, a, b, regA, regB, n):
    for i in range(n):
        if (a[i] == "1"):
            qc.x(regA[n-(i+1)])
        if (b[i] == "1"):
            qc.x(regB[n-(i+1)])
            
def multiplier(qc, regA, regB, regC, auxReg, outputReg, n):
    for i in range(0,2):
        for j in range(n):
            qc.ccx(regA[i], regB[j], auxReg[i + j])
#             qc.measure(auxReg, output)
        cummulativeAdder(qc, auxReg, regC, outputReg, n*2-1)
            # Cleaning Aux
        for j in range(n):
            qc.ccx(regA[i], regB[j], auxReg[i + j])
                
def innerProduct(qc, regAs, regBs, regCs, regAuxs, regOs, m, n):
    for i in range(0, m):
        multiplier(qc, regAs[i], regBs[i], regCs[i], regAuxs[i], regOs[i], n)
    for i in range(1, m):
        cummulativeAdder(qc, regOs[i], regCs[0], regOs[0], n*2)
        
# Requires aux and carry axillery registers
def oracleInner(qc, alpha, beta, i, f, d, aux, carry, oAux, output1, output2, n):
    innerProduct(qc, f, d, carry, aux, oAux)
    # Copy output to an aux for multiplication
    qc.cx(oAux[0], aux[0])
    multiplier(qc, oAux[0], aux[0], carry[0], oAux[1], output2, n)
    # Revert aux0
    qc.cx(oAux[0], aux[0])
    # Revert aux and oAux
    innerProduct(qc, f, d, carry, aux, oAux)

In [67]:
f0 = "11"
f1 = "00"
# f2 = "10"
# f3 = "01"

d0 = "11"
d1 = "00"
# d2 = "10"
# d3 = "01"

iLength = 1
m = 1
n = 2
index = QuantumRegister(iLength, name="i")
indexAux = QuantumRegister(iLength, name="iAux")
f = QuantumRegister(n, name="f")
d = QuantumRegister(n, name="d")

fTotal = []
dTotal = []

regCs = []
regAuxs = []
regOs = []

regCs.append(QuantumRegister(n*2, name="c0"))

regAuxs.append(QuantumRegister(n*2, name="aux0"))

regOs.append(QuantumRegister(n*2+1, name="out0"))


fTotal.append(QuantumRegister(n, name="f0"))
fTotal.append(QuantumRegister(n, name="f1"))
# fTotal.append(QuantumRegister(n, name="f2"))
# fTotal.append(QuantumRegister(n, name="f3"))

dTotal.append(QuantumRegister(n, name="d0"))
dTotal.append(QuantumRegister(n, name="d1"))
# dTotal.append(QuantumRegister(n, name="d2"))
# dTotal.append(QuantumRegister(n, name="d3"))

output = ClassicalRegister(n*2+iLength+1)

# qc = QuantumCircuit(index, indexAux,  f, d, fTotal[0], fTotal[1], fTotal[2], fTotal[3],
#                     dTotal[0], dTotal[1], dTotal[2], dTotal[3], output)

qc = QuantumCircuit(index, indexAux, f, d, fTotal[0], fTotal[1], 
                    dTotal[0], dTotal[1], regCs[0], regAuxs[0], regOs[0], output)
qc.h(index)
initilizeQubits(qc, f0, f1, fTotal[0], fTotal[1], n)
initilizeQubits(qc, d0, d1, dTotal[0], dTotal[1], n)
# initilizeQubits(qc, f2, f3, fTotal[2], fTotal[3], n)
# initilizeQubits(qc, d2, d3, dTotal[2], dTotal[3], n)

load(qc, index, indexAux, f, d, fTotal, dTotal, n, iLength)

innerProduct(qc, [f], [d], regCs, regAuxs, regOs, m, n)

qc.measure(index, output[0:iLength])
qc.measure(regOs[0], output[iLength: n*2+iLength+1])
# qc.measure(f, output[iLength:iLength+n])
# qc.measure(d, output[iLength+n:iLength+2*n])
qc.draw()

3
3


┌───┐                               ░                      ░           »
     i: ┤ H ├──■────────────────────────────░───■────■────■────■───░────────■──»
        └───┘┌─┴─┐┌───┐                     ░   │    │    │    │   ░ ┌───┐┌─┴─┐»
  iAux: ─────┤ X ├┤ X ├──■────■────■────■───░───┼────┼────┼────┼───░─┤ X ├┤ X ├»
             └───┘└───┘┌─┴─┐  │    │    │   ░ ┌─┴─┐  │    │    │   ░ └───┘└───┘»
   f_0: ───────────────┤ X ├──┼────┼────┼───░─┤ X ├──┼────┼────┼───░───■────■──»
                       └─┬─┘  │  ┌─┴─┐  │   ░ └─┬─┘  │  ┌─┴─┐  │   ░   │    │  »
   f_1: ─────────────────┼────┼──┤ X ├──┼───░───┼────┼──┤ X ├──┼───░───┼────┼──»
                         │  ┌─┴─┐└─┬─┘  │   ░   │  ┌─┴─┐└─┬─┘  │   ░   │    │  »
   d_0: ─────────────────┼──┤ X ├──┼────┼───░───┼──┤ X ├──┼────┼───░───■────┼──»
                         │  └─┬─┘  │  ┌─┴─┐ ░   │  └─┬─┘  │  ┌─┴─┐ ░   │    │  »
   d_1: ─────────────────┼────┼────┼──┤ X ├─░───┼────┼────┼──┤ X ├─░───┼────■──»
        ┌───┐            │    │    │  └─┬─┘ ░   │    │    │  └─┬─┘ ░   │    │  »
  f0_0: ┤ X ├────────────■────┼────┼────┼───░───┼────┼────┼────┼───░───┼────┼──»
        ├───┤                 │    │    │   ░   │    │    │    │   ░   │    │  »
  f0_1: ┤ X ├─────────────────┼────■────┼───░───┼────┼────┼────┼───░───┼────┼──»
        └───┘                 │         │   ░   │    │    │    │   ░   │    │  »
  f1_0: ──────────────────────┼─────────┼───░───■────┼────┼────┼───░───┼────┼──»
                              │         │   ░        │    │    │   ░   │    │  »
  f1_1: ──────────────────────┼─────────┼───░────────┼────■────┼───░───┼────┼──»
        ┌───┐                 │         │   ░        │         │   ░   │    │  »
  d0_0: ┤ X ├─────────────────■─────────┼───░────────┼─────────┼───░───┼────┼──»
        ├───┤                           │   ░        │         │   ░   │    │  »
  d0_1: ┤ X ├───────────────────────────■───░────────┼─────────┼───░───┼────┼──»
        └───┘                               ░        │         │   ░   │    │  »
  d1_0: ────────────────────────────────────░────────■─────────┼───░───┼────┼──»
                                            ░                  │   ░   │    │  »
  d1_1: ────────────────────────────────────░──────────────────■───░───┼────┼──»
                                            ░                      ░   │    │  »
  c0_0: ────────────────────────────────────░──────────────────────░───┼────┼──»
                                            ░                      ░   │    │  »
  c0_1: ────────────────────────────────────░──────────────────────░───┼────┼──»
                                            ░                      ░   │    │  »
  c0_2: ────────────────────────────────────░──────────────────────░───┼────┼──»
                                            ░                      ░   │    │  »
  c0_3: ────────────────────────────────────░──────────────────────░───┼────┼──»
                                            ░                      ░ ┌─┴─┐  │  »
aux0_0: ────────────────────────────────────░──────────────────────░─┤ X ├──┼──»
                                            ░                      ░ └───┘┌─┴─┐»
aux0_1: ────────────────────────────────────░──────────────────────░──────┤ X ├»
                                            ░                      ░      └───┘»
aux0_2: ────────────────────────────────────░──────────────────────░───────────»
                                            ░                      ░           »
aux0_3: ────────────────────────────────────░──────────────────────░───────────»
                                            ░                      ░           »
out0_0: ────────────────────────────────────░──────────────────────░───────────»
                                            ░                      ░           »
out0_1: ────────────────────────────────────░──────────────────────░───────────»
                                            ░                      ░           »
out0_2: ───────────────────────────────

NameError: name 'QuantumRegister' is not defined